# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features - adding 'log_' as prefix to avoid problems with feature names starting with numbers
log_names = [f'log_{column}' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

In [3]:
preprocessed.head()

,log_LotArea,log_1stFlrSF,log_GrLivArea,log_SalePrice,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,-0.133185,-0.803295,0.529078,0.559876,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.113403,0.418442,-0.381715,0.212692,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.419917,-0.576363,0.659449,0.733795,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.103311,-0.439137,0.541326,-0.437232,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.878108,0.112229,1.281751,1.014303,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


## Perform stepwise selection

The function for stepwise selection is copied below. Use this provided function on your preprocessed Ames Housing data.

In [4]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()    # should be .idxmax(), not .argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [5]:
# predictors (excluding target variable)
X = preprocessed.drop('log_SalePrice', axis=1)

# target variable
y = preprocessed.log_SalePrice

# apply stepwise_selection
feat_results = stepwise_selection(X, y, verbose=True)
print('---------------------------------------------------------------')
print(f"{len(feat_results)} Selected Features:")
print(feat_results)

Add  log_GrLivArea                  with p-value 1.59847e-243
Add  KitchenQual_TA                 with p-value 1.56401e-67
Add  log_1stFlrSF                   with p-value 7.00069e-48
Add  KitchenQual_Fa                 with p-value 1.70471e-37
Add  Neighborhood_OldTown           with p-value 3.20105e-23
Add  KitchenQual_Gd                 with p-value 4.12635e-21
Add  Neighborhood_Edwards           with p-value 9.05184e-17
Add  Neighborhood_IDOTRR            with p-value 1.10068e-18
Add  log_LotArea                    with p-value 1.71728e-13
Add  Neighborhood_NridgHt           with p-value 7.05633e-12
Add  BldgType_Duplex                with p-value 4.30647e-11
Add  Neighborhood_NAmes             with p-value 2.25803e-09
Add  Neighborhood_SWISU             with p-value 5.40743e-09
Add  Neighborhood_BrkSide           with p-value 8.79638e-10
Add  Neighborhood_Sawyer            with p-value 6.92011e-09
Add  Neighborhood_NoRidge           with p-value 5.87105e-08
Add  Neighborhood_Somer

### Build the final model again in Statsmodels

In [6]:
# import ols
from statsmodels.formula.api import ols

outcome = 'log_SalePrice'
pred_sum = ' + '.join(feat_results)
formula = outcome + ' ~ ' + pred_sum

model = ols(formula=formula, data=preprocessed).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          log_SalePrice   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     269.0
Date:                Wed, 10 Feb 2021   Prob (F-statistic):               0.00
Time:                        18:30:19   Log-Likelihood:                -754.40
No. Observations:                1460   AIC:                             1565.
Df Residuals:                    1432   BIC:                             1713.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.2174      0.164     -1.323      0.186      -0.540       0.105
log_GrLivArea            0.3694      0.015     24.477      0.000       0.340       0.399
KitchenQual_TA          -0.7020      0.055    -12.859      0.000      -0.809      -0.595
log_1stFlrSF             0.1445      0.015      9.645      0.000       0.115       0.174
KitchenQual_Fa          -1.0372      0.087    -11.864      0.000      -1.209      -0.866
Neighborhood_OldTown    -0.8625      0.063    -13.615      0.000      -0.987      -0.738
KitchenQual_Gd          -0.4021      0.050     -8.046      0.000      -0.500      -0.304
Neighborhood_Edwards    -0.7019      0.048    -14.530      0.000      -0.797      -0.607
Neighborhood_IDOTRR     -0.8583      0.097     -8.855      0.000      -1.048      -0.668
log_LotArea              0.1096      0.015      7.387      0.000       0.081       0.139
Neighborhood_NridgHt     0.3854      0.057      6.809      0.000       0.274       0.496
BldgType_Duplex         -0.4073      0.061     -6.678      0.000      -0.527      -0.288
Neighborhood_NAmes      -0.3763      0.038     -9.981      0.000      -0.450      -0.302
Neighborhood_SWISU      -0.6263      0.089     -7.020      0.000      -0.801      -0.451
Neighborhood_BrkSide    -0.5641      0.066     -8.493      0.000      -0.694      -0.434
Neighborhood_Sawyer     -0.4026      0.055     -7.342      0.000      -0.510      -0.295
Neighborhood_NoRidge     0.4347      0.070      6.221      0.000       0.298       0.572
Neighborhood_StoneBr     0.4538      0.087      5.226      0.000       0.283       0.624
Neighborhood_MeadowV    -0.6622      0.118     -5.592      0.000      -0.895      -0.430
SaleType_New             0.1483      0.044      3.388      0.001       0.062       0.234
Neighborhood_BrDale     -0.4733      0.123     -3.839      0.000      -0.715      -0.231
MSZoning_RM              1.0820      0.147      7.363      0.000       0.794       1.370
MSZoning_RL              0.9916      0.156      6.356      0.000       0.686       1.298
MSZoning_FV              1.2052      0.165      7.284      0.000       0.881       1.530
MSZoning_RH              0.8503      0.189      4.490      0.000       0.479       1.222
Neighborhood_NWAmes     -0.2055      0.054     -3.837      0.000      -0.311      -0.100
Neighborhood_Mitchel    -0.1943      0.065     -3.004      0.003      -0.321      -0.067
Neighborhood_SawyerW    -0.1666      0.058     -2.862      0.004      -0.281      -0.052
==============================================================================
Omnibus:                      295.535   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1270.571
Skew:                          -0.903   Prob(JB):                    1.26e-276
Kurtosis

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [7]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

Fit the linear regression model again using the 5 selected columns

In [8]:
# Your code here
# number of features to select
p = 5

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=p)
selector = selector.fit(X, y)

In [9]:
# 5 selected features
list(X.columns[selector.support_])

['MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Neighborhood_NoRidge']

In [10]:
# R-Squared
selector.score(X, y)

0.23943418177114228

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [11]:
# Your code here
y_hat = selector.predict(X)    

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [12]:
# create a function that calculates sum of squares
def sum_of_squares(y1, y2):
    y1 = np.array(y1)
    y2 = np.array(y2)
    
    ss = ((y1 - y2)**2).sum()
    return ss

In [13]:
# total number of data points
n = len(y)
# number of selected features
p = 5

# create a baseline array 
y_bar = np.ones(n) * np.mean(y)

# calculate ssr and sse using sum_of_squares() function above
ssr = sum_of_squares(y, y_hat)
sse = sum_of_squares(y, y_bar)

# calculate r_squared & adj_r_squared
r_squared = 1 - (ssr/sse)
adj_r_squared = 1 - (1 - r_squared) * ((n-1)/(n-p-1))

# print results
print(f'R-Squared: {round(r_squared, 6)}')
print(f'Adjusted R-Squared: {round(adj_r_squared, 6)}')

# r_squared is 0.239434  
# adjusted_r_squared is 0.236818

R-Squared: 0.239434
Adjusted R-Squared: 0.236819


In [14]:
# Feature selecting using stepwise_selection() and StatsModels produces significantly higher R-Squared as well as Adjusted R-Squared.

## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

In [15]:
def forward_selection(X, y, 
                      initial_list=[],  
                      verbose=True):
    """ 
    Perform a forward feature selection 
    based on Adjusted R-Squared from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    
    """
    included = list(initial_list)
    best_adj_r2 = 0.0
    current_adj_r2 = 0.0
    while (len(included) < len(X.columns)) and (current_adj_r2 == best_adj_r2):
        # forward step
        excluded = sorted(list(set(X.columns)-set(included)))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            current_adj_r2 = model.rsquared_adj
            
            if current_adj_r2 > best_adj_r2:
                best_adj_r2 = current_adj_r2
                included.append(new_column)
                if verbose:
                    print(f'Add: {new_column}')

    return included, current_adj_r2

In [16]:
# predictors (excluding target variable)
X = preprocessed.drop('log_SalePrice', axis=1)

# target variable
y = preprocessed.log_SalePrice

# apply forward_selection
results = forward_selection(X, y, verbose=True)
print('---------------------------------------------------------------')
print(f'Selected Features: {results[0]}')
print(f'Current Best Adjusted R-Squared: {results[1]}')

Add: BldgType_2fmCon
Add: BldgType_Duplex
Add: BldgType_Twnhs
Add: KitchenQual_Fa
Add: KitchenQual_Gd
Add: KitchenQual_TA
Add: MSZoning_FV
Add: MSZoning_RL
Add: MSZoning_RM
Add: Neighborhood_Blueste
Add: Neighborhood_BrkSide
Add: Neighborhood_ClearCr
Add: Neighborhood_Crawfor
Add: Neighborhood_Edwards
Add: Neighborhood_Gilbert
Add: Neighborhood_IDOTRR
Add: Neighborhood_MeadowV
Add: Neighborhood_NAmes
Add: Neighborhood_NWAmes
Add: Neighborhood_NoRidge
Add: Neighborhood_NridgHt
Add: Neighborhood_OldTown
Add: Neighborhood_SWISU
Add: Neighborhood_Sawyer
Add: Neighborhood_SawyerW
Add: Neighborhood_Somerst
Add: Neighborhood_StoneBr
Add: Neighborhood_Timber
Add: Neighborhood_Veenker
Add: SaleType_CWD
Add: SaleType_Con
Add: SaleType_New
Add: SaleType_WD
Add: log_1stFlrSF
Add: log_GrLivArea
Add: log_LotArea
Add: MSZoning_RH
Add: Neighborhood_BrDale
Add: Neighborhood_CollgCr
Add: Neighborhood_Mitchel
Add: SaleType_ConLD
Add: Street_Pave
-----------------------------------------------------------

In [17]:
len(results[0])

42

In [18]:
# Fit model using these forward selected features
outcome = 'log_SalePrice'
pred_sum = ' + '.join(results[0])
formula = outcome + ' ~ ' + pred_sum

model2 = ols(formula=formula, data=preprocessed).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          log_SalePrice   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     175.6
Date:                Wed, 10 Feb 2021   Prob (F-statistic):               0.00
Time:                        18:30:20   Log-Likelihood:                -738.66
No. Observations:                1460   AIC:                             1563.
Df Residuals:                    1417   BIC:                             1791.
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.1862      0.244     -0.763      0.446      -0.665       0.293
BldgType_2fmCon         -0.1743      0.079     -2.215      0.027      -0.329      -0.020
BldgType_Duplex         -0.4157      0.061     -6.773      0.000      -0.536      -0.295
BldgType_Twnhs          -0.1053      0.081     -1.297      0.195      -0.265       0.054
KitchenQual_Fa          -0.9999      0.088    -11.356      0.000      -1.173      -0.827
KitchenQual_Gd          -0.3832      0.050     -7.657      0.000      -0.481      -0.285
KitchenQual_TA          -0.6698      0.055    -12.169      0.000      -0.778      -0.562
MSZoning_FV              1.0708      0.193      5.557      0.000       0.693       1.449
MSZoning_RL              1.0126      0.161      6.301      0.000       0.697       1.328
MSZoning_RM              1.1044      0.151      7.296      0.000       0.807       1.401
Neighborhood_Blueste     0.0777      0.308      0.253      0.801      -0.526       0.681
Neighborhood_BrkSide    -0.5976      0.109     -5.497      0.000      -0.811      -0.384
Neighborhood_ClearCr    -0.1774      0.125     -1.415      0.157      -0.423       0.069
Neighborhood_Crawfor    -0.0446      0.106     -0.419      0.675      -0.253       0.164
Neighborhood_Edwards    -0.7206      0.098     -7.328      0.000      -0.913      -0.528
Neighborhood_Gilbert    -0.0587      0.102     -0.575      0.565      -0.259       0.142
Neighborhood_IDOTRR     -0.9034      0.133     -6.811      0.000      -1.164      -0.643
Neighborhood_MeadowV    -0.6652      0.142     -4.690      0.000      -0.943      -0.387
Neighborhood_NAmes      -0.3999      0.093     -4.277      0.000      -0.583      -0.216
Neighborhood_NWAmes     -0.2292      0.102     -2.249      0.025      -0.429      -0.029
Neighborhood_NoRidge     0.4009      0.112      3.585      0.000       0.182       0.620
Neighborhood_NridgHt     0.3894      0.100      3.876      0.000       0.192       0.586
Neighborhood_OldTown    -0.8794      0.110     -8.019      0.000      -1.094      -0.664
Neighborhood_SWISU      -0.6560      0.122     -5.370      0.000      -0.896      -0.416
Neighborhood_Sawyer     -0.4337      0.102     -4.239      0.000      -0.634      -0.233
Neighborhood_SawyerW    -0.1980      0.104     -1.911      0.056      -0.401       0.005
Neighborhood_Somerst     0.1337      0.126      1.061      0.289      -0.113       0.381
Neighborhood_StoneBr     0.4463      0.118      3.774      0.000       0.214       0.678
Neighborhood_Timber      0.0419      0.113      0.369      0.712      -0.181       0.265
Neighborhood_Veenker     0.1529      0.156      0.982      0.326      -0.153       0.458
SaleType_CWD             0.2115      0.213      0.994      0.320      -0.206       0.629
SaleType_Con             0.5655      0.302      1.874    

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 